https://drive.google.com/drive/folders/1peThKQigJkn6CTFpIfI3jfkgH5s65EeE -- данные

Используйте реализацию многослойного персептрона из ЛР 4. Реализуйте сверточный слой (прямое и обратное распространение). Соберите сверточную сеть. Попробуйте обучить классификатор кошек и собак с использованием собственной реализации сверточной нейронной сети. Вам также потребуется реализовать слой для преобразования многомерных массивов данных в одномерные.

Надо сделать:
1. Переписать персептрон (полностью);
2. Написать Адама;
3. Написать сверточный слой (прямое и обратное распространение);
4. Написать слой для стягивания тензора в вектор;
5. Обучить классификатор кошек и собак на своей нейронке;
6. *Написать пулинговый слой 

In [1]:
from pathlib import Path
import os
import sys

sys.path.append(str(Path(os.getcwd()).parent))

In [82]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [107]:
from sklearn.metrics import classification_report

# 1. Решение задачи классификации кошек и собак

## 1.1 Загрузка датасета и аугментация

In [83]:
train_ds, test_ds = tf.keras.utils.image_dataset_from_directory(
    directory="../data/cats&dogs", 
    validation_split=0.2, 
    subset="both", 
    seed=13, 
    image_size=(200, 200), # СНАЧАЛА ВЫСОТА, ПОТОМ ШИРИНА
    batch_size=32
)

Found 2000 files belonging to 2 classes.
Using 1600 files for training.
Using 400 files for validation.


In [85]:
data_augmentation_flip = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
])

In [86]:
train_ds = train_ds.map(
    lambda img, label: (data_augmentation_flip(img), label)
)

![cat](..\memes\0_yV46bI9fS12FXrh-.png)

## 1.2 Построение модели

In [88]:
inputs = tf.keras.Input(shape=(200, 200, 3))

x = tf.keras.layers.Rescaling(1. / 255)(inputs)

x = tf.keras.layers.Conv2D(64, 3, strides=2, padding="same")(x)
x = tf.keras.layers.MaxPooling2D(3, strides=2, padding="same")(x)

x = tf.keras.layers.Conv2D(128, 4, strides=2, padding="same")(x)
x = tf.keras.layers.MaxPooling2D(3, strides=2, padding="same")(x)

x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Dense(64, activation="relu")(x)
outputs = tf.keras.layers.Dense(2, activation="softmax")(x)

classification_model = tf.keras.Model(inputs, outputs)

In [89]:
classification_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [90]:
classification_model.fit(train_ds, epochs=5, validation_data=test_ds)

Epoch 1/5
50/50 [==============================] - 19s 363ms/step - loss: 0.8913 - accuracy: 0.5081 - val_loss: 0.6776 - val_accuracy: 0.5350
Epoch 2/5
50/50 [==============================] - 18s 353ms/step - loss: 0.6752 - accuracy: 0.5756 - val_loss: 0.6666 - val_accuracy: 0.5575
Epoch 3/5
50/50 [==============================] - 18s 355ms/step - loss: 0.6413 - accuracy: 0.6525 - val_loss: 0.6463 - val_accuracy: 0.6025
Epoch 4/5
50/50 [==============================] - 18s 363ms/step - loss: 0.6388 - accuracy: 0.6356 - val_loss: 0.6330 - val_accuracy: 0.6300
Epoch 5/5
50/50 [==============================] - 18s 356ms/step - loss: 0.6304 - accuracy: 0.6494 - val_loss: 0.6710 - val_accuracy: 0.5925


In [104]:
img = tf.keras.preprocessing.image.load_img(
    "../data/cats&dogs/cats/2.jpg", target_size=(200, 200)
)

img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

print("Это собака" if np.argmax(classification_model.predict(img_array)) else "Это кот")

1/1 [==============================] - 0s 21ms/step
Это кот


In [146]:
pred = [np.argmax(pred) for pred in classification_model.predict(test_ds)]

y_test = np.array([])

images, labels = tuple(zip(*test_ds))
for i in labels:
    y_test = np.append(y_test, i)

print(classification_report(y_test, pred))

13/13 [==============================] - 5s 213ms/step
              precision    recall  f1-score   support

         0.0       0.50      0.89      0.64       191
         1.0       0.67      0.20      0.31       209

    accuracy                           0.53       400
   macro avg       0.59      0.55      0.48       400
weighted avg       0.59      0.53      0.47       400



# 2. Работа с уже обученными моделями

## 2.1 Загрузка моделей InceptionV3 и VGG19

In [ ]:
vgg = tf.keras.applications.VGG19(weights='imagenet', include_top=True)
# vgg.summary()

In [178]:
iv3 = tf.keras.applications.InceptionV3(weights='imagenet', include_top=True)
# iv3.summary()

## 2.2 Feature-extraction и Fine-tuning для VGG19

In [ ]:
train_ds, test_ds = tf.keras.utils.image_dataset_from_directory(
    directory="../data/objects", 
    validation_split=0.2, 
    subset="both", 
    seed=9, 
    image_size=(224, 224),
    batch_size=32
)

In [154]:
input = vgg.input
my_layer = tf.keras.layers.Dense(3, activation='softmax')
output = my_layer(vgg.layers[-2].output)

my_vgg = tf.keras.Model(input, output)

In [160]:
pred = [np.argmax(pred) for pred in my_vgg.predict(test_ds)]

y_test = np.array([])

images, labels = tuple(zip(*test_ds))
for i in labels:
    y_test = np.append(y_test, i)

print(classification_report(y_test, pred))

2/2 [==============================] - 4s 1s/step
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        15
         1.0       1.00      0.07      0.13        14
         2.0       0.35      1.00      0.52        14

    accuracy                           0.35        43
   macro avg       0.45      0.36      0.22        43
weighted avg       0.44      0.35      0.21        43



In [ ]:
my_vgg.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [163]:
for layer in my_vgg.layers:
    layer.trainable = False
my_vgg.layers[-1].trainable = True

In [ ]:
my_vgg.fit(train_ds, epochs=5, validation_data=test_ds)

In [166]:
pred = [np.argmax(pred) for pred in my_vgg.predict(test_ds)]

y_test = np.array([])

images, labels = tuple(zip(*test_ds))
for i in labels:
    y_test = np.append(y_test, i)

print(classification_report(y_test, pred))

2/2 [==============================] - 4s 1s/step
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        15
         1.0       1.00      1.00      1.00        14
         2.0       1.00      1.00      1.00        14

    accuracy                           1.00        43
   macro avg       1.00      1.00      1.00        43
weighted avg       1.00      1.00      1.00        43



In [167]:
my_vgg.layers[-2].trainable = True

In [168]:
my_vgg.fit(train_ds, epochs=5, validation_data=test_ds)

Epoch 1/5
6/6 [==============================] - 20s 3s/step - loss: 2.8583e-04 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 2/5
6/6 [==============================] - 19s 3s/step - loss: 1.8003e-04 - accuracy: 1.0000 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 3/5
6/6 [==============================] - 20s 3s/step - loss: 1.2785e-04 - accuracy: 1.0000 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 4/5
6/6 [==============================] - 21s 4s/step - loss: 1.0499e-04 - accuracy: 1.0000 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 5/5
6/6 [==============================] - 20s 3s/step - loss: 8.9217e-05 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy: 1.0000


In [169]:
pred = [np.argmax(pred) for pred in my_vgg.predict(test_ds)]

y_test = np.array([])

images, labels = tuple(zip(*test_ds))
for i in labels:
    y_test = np.append(y_test, i)

print(classification_report(y_test, pred))

2/2 [==============================] - 4s 1s/step
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        15
         1.0       1.00      1.00      1.00        14
         2.0       1.00      1.00      1.00        14

    accuracy                           1.00        43
   macro avg       1.00      1.00      1.00        43
weighted avg       1.00      1.00      1.00        43



## 2.3 Feature-extraction и Fine-tuning для InceptionV3

In [179]:
train_ds, test_ds = tf.keras.utils.image_dataset_from_directory(
    directory="../data/objects", 
    validation_split=0.2, 
    subset="both", 
    seed=9, 
    image_size=(299, 299),
    batch_size=32
)

Found 216 files belonging to 3 classes.
Using 173 files for training.
Using 43 files for validation.


In [180]:
input = iv3.input
my_layer = tf.keras.layers.Dense(3, activation='softmax')
output = my_layer(iv3.layers[-2].output)

my_iv3 = tf.keras.Model(input, output)

In [181]:
pred = [np.argmax(pred) for pred in my_iv3.predict(test_ds)]

y_test = np.array([])

images, labels = tuple(zip(*test_ds))
for i in labels:
    y_test = np.append(y_test, i)

print(classification_report(y_test, pred))

2/2 [==============================] - 3s 452ms/step
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        15
         1.0       0.00      0.00      0.00        14
         2.0       0.33      1.00      0.49        14

    accuracy                           0.33        43
   macro avg       0.11      0.33      0.16        43
weighted avg       0.11      0.33      0.16        43



c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [182]:
my_iv3.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [185]:
for layer in my_iv3.layers:
    layer.trainable = False
my_iv3.layers[-1].trainable = True

In [186]:
my_iv3.fit(train_ds, epochs=5, validation_data=test_ds)

Epoch 1/5
6/6 [==============================] - 60s 8s/step - loss: 0.2686 - accuracy: 0.8382 - val_loss: 5.5892 - val_accuracy: 0.4651
Epoch 2/5
6/6 [==============================] - 47s 8s/step - loss: 0.0653 - accuracy: 0.9884 - val_loss: 24.0082 - val_accuracy: 0.3953
Epoch 3/5
6/6 [==============================] - 45s 7s/step - loss: 0.1054 - accuracy: 0.9711 - val_loss: 804.2016 - val_accuracy: 0.3256
Epoch 4/5
6/6 [==============================] - 43s 7s/step - loss: 0.2037 - accuracy: 0.9480 - val_loss: 2943.6809 - val_accuracy: 0.2791
Epoch 5/5
6/6 [==============================] - 44s 7s/step - loss: 0.0747 - accuracy: 0.9884 - val_loss: 866.7255 - val_accuracy: 0.3488


In [188]:
pred = [np.argmax(pred) for pred in my_iv3.predict(test_ds)]

y_test = np.array([])

images, labels = tuple(zip(*test_ds))
for i in labels:
    y_test = np.append(y_test, i)

print(classification_report(y_test, pred))

2/2 [==============================] - 2s 431ms/step
              precision    recall  f1-score   support

         0.0       0.35      1.00      0.52        15
         1.0       0.00      0.00      0.00        14
         2.0       0.00      0.00      0.00        14

    accuracy                           0.35        43
   macro avg       0.12      0.33      0.17        43
weighted avg       0.12      0.35      0.18        43



c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bugae\MachineLearning-Labs\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
